In [ ]:
import os
from pandas import read_table
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr, Metadata
from mutperiodpy.GeneratePlotnineFigures import parseAndPlotPeriodicity

jiangDataDirectory = os.path.join(getDataDir(), "Jiang_BPDE_damage_maps")

dataSets = ["BEAS-2B_2uM_BPDE_cell_24h"]

In [ ]:
SINGLE_NUC = 1
NUC_GROUP = 2

jiangRawCounts = dict()
jiangNormalizedCounts = dict()

TFBS_Names = ["hg19_CTCF_known_all", "hg19_ETS_known_all", "hg19_GABPA_known_all", "hg19_SP1_known_all"]

for dataSet in dataSets:

    jiangRawCounts[dataSet] = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
    jiangNormalizedCounts[dataSet] = {SINGLE_NUC:dict(), NUC_GROUP:dict()}

    for nucleosomeCountsFilePath in getFilesInDirectory(jiangDataDirectory, DataTypeStr.rawNucCounts+".tsv"):

        countsMetadata = Metadata(nucleosomeCountsFilePath)
        if countsMetadata.nucPosName not in TFBS_Names: continue

        if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
        else: radius = SINGLE_NUC

        jiangRawCounts[dataSet][radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

    for nucleosomeCountsFilePath in getFilesInDirectory(jiangDataDirectory, DataTypeStr.normNucCounts+".tsv"):

        countsMetadata = Metadata(nucleosomeCountsFilePath)
        if countsMetadata.nucPosName not in TFBS_Names: continue

        if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
        else: radius = SINGLE_NUC

        jiangNormalizedCounts[dataSet][radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

In [ ]:
for dataSet in dataSets:
    for radius in jiangRawCounts[dataSet]:
        for TFBS_Name in jiangRawCounts[dataSet][radius]:
            plot = parseAndPlotPeriodicity(jiangRawCounts[dataSet][radius][TFBS_Name], dataCol = "Both_Strands_Counts",
                                           smoothTranslational = False, nucRepLen = 2000,
                                           title = f"Jiang {dataSet} {TFBS_Name} Raw Counts", yAxisLabel = "Damage Reads")
            print(plot)

In [ ]:
for dataSet in dataSets:
    for radius in jiangNormalizedCounts[dataSet]:
        for TFBS_Name in jiangNormalizedCounts[dataSet][radius]:
            plot = parseAndPlotPeriodicity(jiangNormalizedCounts[dataSet][radius][TFBS_Name], dataCol = "Normalized_Both_Strands",
                                           smoothTranslational = True, nucRepLen = 2000,
                                           title = f"Jiang {dataSet} {TFBS_Name} Naked Normalized", yAxisLabel = "Cellular/Naked Damage Reads")
            print(plot)